In [13]:
%matplotlib inline

import math
import random
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

### Problem 1
  - linear 레이어의 weight를 normal distribution으로 초기화 하도록 구현
      - hint: "normal"을 tensorflow 홈페이지에서 검색
  - 약 92% accuracy를 가지도록 learning rate 찾기
  
### Problem 2
  - relu 레이어 구현
      - $f(x)=x~$ (if $x\ge0$) and $f(x)=0~$ (if $x<0$)
      - hint: "relu"을 tensorflow 홈페이지에서 검색
  - 약 98% accuracy를 가지도록 파라미터 찾기

### Problem 3
  - tensorflow.contrib.layers 유틸리티 사용해보기
  - use_contribute=True로 바꾼 후, linearLayer 함수를 tensorflow.contrib.layers에 있는 함수로 구현
      - https://www.tensorflow.org/api_guides/python/contrib.layers      

### Problem 4
  - hidden 레이어를 하나 더 추가하고, 성능이 나오는 파라미터를 찾기
  - 마지막에 Linear layer (10dim -> 10dim) 추가


### Load MNIST data

In [14]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets('MNIST_data', one_hot=True)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


### Model Construction Functions

In [43]:
# Functions of layer definition
def linearLayer(inp, inp_dim, out_dim, with_relu=False, \
                prefix='hidden', use_contribute=False):
    """ Linear layer
    Args:
        inp: input data, [batch_size, inp_dim]
        inp_dim: input dimension
        out_dim: output dimension
    Returns:
        out: Output tensor with the computed logits, [batch_size, out_dim]
             return (inp * weight + bias)
    """
    
    with tf.name_scope(prefix):
        if not use_contribute :
            """ Problem 1
            TODO: initializing weight using normal distribution, mean (1.0) and std (0.1) 
            """
            weight = tf.Variable(TODO_FillHere,name='weights')
            bias = tf.Variable(tf.zeros([out_dim]),name='biases')

            out = tf.matmul(inp, weight) + bias
            if with_relu : 
                """ Problem 2
                TODO: Implement relu operation
                """
                out = tf.nn.relu(out)
        else :
            """ Problem 3
            TODO: Using tf.contrib.layers utility
            """
            TODO = True
    return out

# Functions of model construction
def buildInputHolders():
    """ define placeholders for model inputs
    Returns:
        img_holder: images placeholder
        label_hodler: labels placeholder
    """
    img_holder = tf.placeholder(tf.float32, [None, 28*28])
    label_holder = tf.placeholder(tf.float32, [None, 10])
    
    return img_holder, label_holder

""" Problem 4
TODO: adding additional linear layer
"""
def buildInference(images, img_dim, label_num, hidden1_units, hidden2_units):
    """Build the MNIST model up to where it may be used for inference.
    Args:
        images: Images placeholder, from inputs().
        hidden1_units: Size of the first hidden layer.
        hidden2_units: Size of the second hidden layer.
    Returns:
        pred: Output tensor with the computed logits.
    """
    # Hidden 1
    hidden1 = linearLayer(images, img_dim, hidden1_units, \
                          with_relu=True, prefix='hidden1')
    # Hidden 2
    hidden2 = linearLayer(hidden1, hidden1_units, \
                          hidden2_units, with_relu=True, prefix='hidden2')
    
    # Linear
    pred = linearLayer(hidden2, hidden2_units, \
                       label_num, with_relu=False, prefix='linear')
    
    return tf.nn.softmax(pred)

def buildLoss(preds, labels):
    """Calculates the loss from the predictions and the labels.
    Args:
        preds  : prediction tensor, [batch_size, NUM_CLASSES]
        labels : labels tensor, [batch_size]
    Returns:
        loss : loss tensor of type float
    """
    cross_entropy_loss = -tf.reduce_sum(labels * tf.log(preds), reduction_indices=[1])
    return tf.reduce_mean(cross_entropy_loss)

def buildAccuracy(pred, labels):
    """Calculates the accuracy from the predictions and the labels.
    Args:
        preds  : prediction tensor, [batch_size, NUM_CLASSES]
        labels : labels tensor, [batch_size]
    Returns:
        accuracy : accuracy tensor of type float
    """
    correct_prediction = tf.equal(tf.argmax(pred,1), tf.argmax(labels,1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
    
    return accuracy

def buildModel(params):
    """ Build Model
    Args:
        params : dictionary data for parameters {
            'batch_size' : batch size
            'lr' : learning rate
            }
    Returns:
        imgs : input images of model
        labels : input labels of model
        train_step : one step operation for training
        loss : Loss tensor
        acc : accuracy
    """
    imgs, labels = buildInputHolders()
    preds = buildInference(imgs, 28*28, 10, 128, 32)
    loss = buildLoss(preds, labels)
    acc = buildAccuracy(preds, labels)
    
    global_step = tf.Variable(.0, trainable=False)
    optimizer = tf.train.GradientDescentOptimizer(params['lr'])
    train_step = optimizer.minimize(loss, global_step=global_step)
    
    return imgs, labels, train_step, loss, acc

### Main Function

In [44]:
# define parameters
params = {}
params['batch_size'] = 100
params['lr'] = 0.5
params['total_batch'] = int(mnist.train.num_examples/params['batch_size'])

# build model
imgs, labels, train_step, loss, acc = buildModel(params)

In [45]:
batch_size = params['batch_size']
total_batch = params['total_batch']

init = tf.global_variables_initializer()

with tf.Session() as sess:
    sess.run(init)
    for epoch in range(10):
        total_cost = 0

        for i in range(total_batch):
            batch_xs, batch_ys = mnist.train.next_batch(batch_size)
            _, cost_val = sess.run([train_step, loss], \
                                   feed_dict={imgs: batch_xs, labels: batch_ys})
            total_cost += cost_val

        print( "Epoch:", "%04d" % (epoch + 1), \
            "Avg. cost =", '{:.3f}'.format(total_cost / total_batch))

    # 테스트 데이터에 대한 정확도
    print("Test Accuracy: ", sess.run(acc, \
                    feed_dict={imgs: mnist.test.images, labels: mnist.test.labels}))

Epoch: 0001 Avg. cost = 0.449
Epoch: 0002 Avg. cost = 0.196
Epoch: 0003 Avg. cost = 0.142
Epoch: 0004 Avg. cost = 0.112
Epoch: 0005 Avg. cost = 0.093
Epoch: 0006 Avg. cost = 0.077
Epoch: 0007 Avg. cost = 0.067
Epoch: 0008 Avg. cost = 0.057
Epoch: 0009 Avg. cost = 0.050
Epoch: 0010 Avg. cost = 0.045
Test Accuracy:  0.9744
